In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/WDI

Mounted at /content/drive
/content/drive/My Drive/WDI


In [ ]:
!pip install --upgrade pandas --user
!pip install python-Levenshtein

In [3]:
import pandas as pd
import numpy as np

## Fix "out of memory" ： Data set reduction

1. For Netflix, keep the columns for matching
2. For imdb, keep the columns for matching, reduce rows further
3. Row reduction: add column sim recording the first similarity bigger than threshold, that is 0.3. delete random rows from those ones whose sim are 0.

Because the use of block key, there definitely some mistake deletion

In [74]:
# NETFLIX:only keep the columns for matching
from utils import to_xml

columns = ['id', 'title',  'release_year', 'genre', 'actor_names', 'language', 'writer',  'director', 'imdb_score']

netflix_mini = netflix_df[netflix_df.series_or_movie == 'Movie']
netflix_mini = netflix_mini[columns]
display(netflix_mini.head())
netflix_mini.to_xml(filename='data/NEW/netflix.xml')

,id,title,release_year,genre,actor_names,language,writer,director,imdb_score
1,netflix_1,HOW TO BUILD A GIRL,2020,[Comedy],"[Cleo, Beanie Feldstein, Dónal Finn, Paddy Con...",[English],[Caitlin Moran],[Coky Giedroyc],5.8
2,netflix_2,Centigrade,2020,"[Drama, Thriller]","[Genesis Rodriguez, Vincent Piazza]",[English],"[Daley Nixon, Brendan Walsh]",[Brendan Walsh],4.3
4,netflix_4,Moxie,2011,"[Drama, Short, Animation]",[Ragga Gudrun],[English],NaN,[Stephen Irwin],6.3
5,netflix_5,The Con-Heartist,2020,"[Comedy, Romance]","[Nadech Kugimiya, Thiti Mahayotaruk, Pimchanok...",[Thai],"[Thodsapon Thiptinnakorn, Pattaranad Bhiboonsa...",[Mez Tharatorn],7.4
8,netflix_8,Only a Mother,1949,[Drama],"[Ulf Palme, Eva Dahlbeck, Ragnar Falck, Hugo B...",[Swedish],[Ivar Lo-Johansson],[Alf Sjöberg],6.7


In [55]:
# imdb: reduce columns
imdb_mini = imdb_df[columns]
imdb_mini.title = imdb_mini.title.str.replace('&', 'and')
display(imdb_mini.head())
#imdb_mini.to_xml(filename='data/NEW/imdb.xml')

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,id,title,release_year,genre,actor_names,language,writer,director,imdb_score
0,imdb_0,Miss Jerry,1894,[Romance],"[William Courtenay, Blanche Bayliss, Chauncey ...",NaN,[Alexander Black],[Alexander Black],5.9
1,imdb_1,The Story of the Kelly Gang,1906,"[Biography, Drama, Crime]","[Ollie Wilson, Sam Crewes, Norman Campbell, Wi...",NaN,[Charles Tait],[Charles Tait],6.1
2,imdb_2,Den sorte drøm,1911,[Drama],"[Emil Albes, Asta Nielsen, Hugo Flink, Gunnar ...",NaN,"[Urban Gad, Gebhard Schätzler-Perasini]",[Urban Gad],5.8
3,imdb_3,Cleopatra,1912,"[History, Drama]","[Mr. Brady, Miss Robson, Miss Fielding, Mr. Pa...",[English],[Victorien Sardou],[Charles L. Gaskill],5.2
4,imdb_4,L'Inferno,1911,"[Fantasy, Drama, Adventure]","[Salvatore Papa, Giuseppe de Liguoro, Attilio ...",[Italian],[Dante Alighieri],"[Adolfo Padovan, Francesco Bertolini]",7.0


In [ ]:
# TEMP CODE DO NOT USE ANY MORE
import xml.etree.ElementTree as ET

# load xml
imdb_tree = ET.parse('data/NEW/imdb.xml')

imdb_id = imdb_tree.getroot().findall('./movie/id')

ids = [id.text for id in imdb_id]

tem = pd.DataFrame({'matching_id': ids})
imdb_mini = pd.merge(imdb_mini, tem, how='inner', left_on='id', right_on='matching_id').drop(['matching_id'], axis=1)
imdb_mini.to_xml(filename='data/NEW/imdb.xml')


In [ ]:
# Add auxiliary columns
imdb_mini['sim'] =   # set the column to the first similarity bigger than threshold, that is 0.3
netflix_mini['block_key'] = netflix_mini.title.apply(lambda x: x[:2])
imdb_mini['block_key'] = imdb_mini.title.apply(lambda x: x[:2])

In [ ]:
# auxiliary function for calculating the similarity between list. 
# invoked in dataset reduce and gold stantard

def match_list_attribute(x, y, denominator='min'):
  sim = 0.5 # if NaN, suppose 50% probability the same
  if isinstance(x, list) and isinstance(y, list):
    match_count = len(set(x).intersection(set(y)))
    if match_count == 0:
      return 0

    item_count = 0
    if denominator == 'min':
      item_count = min(len(x), len(y))
    elif denominator == 'max':
      item_count = max(len(x), len(y))
    else:
      item_count = len(x) + len(y) # sum

    sim = match_count / item_count
  return sim

In [ ]:
# delete movies that difintely not match to netflix
# step 1: add sim columns
from Levenshtein import distance

threshold = 0.3 # one in netflix, sim > 0.3, set the sim

# set sim value
for i, row_x in imdb_mini.iterrows():
  for j, row_y in netflix_mini[netflix_mini['block_key'] == row_x['block_key']].iterrows():
    # match attribute
    # year -- directly pass
    if row_x['release_year'] == 'TV Movie 2019':
      imdb_mini.loc[i, 'release_year'] = 2019
      break
    if abs(int(row_x['release_year']) - row_y['release_year']) > 5:
      continue
    
    dist = distance(row_x['title'], row_y['title'])
    sim_title = 1 - dist / max(len(row_x['title']), len(row_y['title']))

    sim_actor = match_list_attribute(row_x['actor_names'], row_y['actor_names'])
    sim_writer = match_list_attribute(row_x['writer'], row_y['writer'])
    sim_director = match_list_attribute(row_x['director'], row_y['director'])
    
    # final similarity
    sim = sim_title * 0.7 + sim_actor * 0.1 + sim_director * 0.1 + sim_writer * 0.1

    # match the first one--> match the max one
    if sim > threshold:
      imdb_mini.loc[i, 'sim'] = sim
      break

In [ ]:
# check the row number of sim = 0, sample the rows we wanna drop
drop_index = imdb_mini[imdb_mini.sim == 0].sample(70000).index

imdb_mini_2 = imdb_mini.drop(drop_index)
imdb_mini_2 = imdb_mini_2[columns]
imdb_mini_2.to_xml(filename='data/NEW/imdb.xml')

### generate reduced complete imdb xml

In [ ]:
import xml.etree.ElementTree as ET

# load xml
imdb_tree = ET.parse('data/imdb_n.xml')
imdb_root = imdb_tree.getroot()

# imdb id
imdb_c_tree = ET.parse('data/NEW/imdb.xml')
movies = imdb_c_tree.getroot().findall('./movie/id/..')

# construct new xml
root = ET.Element("movies")
root.extend(movies)

tree = ET.ElementTree(root)
tree.write("data/NEW/reduced_imdb.xml", encoding="utf-8", xml_declaration=True)


## delete all the series in Netflix

In [ ]:
import xml.etree.ElementTree as ET

# load xml
n_tree = ET.parse('data/netflix_n.xml')
n_root = n_tree.getroot()

# id of series
netflix = pd.read_csv('data/netflix.csv')
netflix_ids = netflix[netflix.series_or_movie == 'Movie'].id.tolist()

# construct new xml
root = ET.Element("movies")
for id in netflix_ids:
  root.append(n_root.find(f'.//id[.="{id}"]/..'))

tree = ET.ElementTree(root)
tree.write("data/NEW/netflix_movie.xml", encoding="utf-8", xml_declaration=True)

# Post-precessing of XML: delete the nan line in xml

In [ ]:
new_lines = []

with open('data/streaming.xml', 'r', encoding='UTF-8') as f:
  while True:
    line = f.readline()
    if not line:
      break
    if '>nan<' not in line:
      new_lines.append(line)

with open('data/streaming_without_nan.xml', 'w', encoding='UTF-8') as f:
  for line in new_lines:
    f.write(line)

## Code to assist in the construction of gold standard of Data Fusion

print the movies in 3 xml


In [67]:
# get xml tree
import xml.etree.ElementTree as ET

imdb_tree = ET.parse('data/imdb_n.xml')
imdb_root = imdb_tree.getroot()

netflix_tree = ET.parse('data/netflix.xml')
netflix_root = netflix_tree.getroot()

streaming_tree = ET.parse('data/streaming.xml')
streaming_root = streaming_tree.getroot()

In [68]:
def print_xml(xml_string):
  xml_list = str(xml_string).strip('b"\' ').split('\\n')
  for item in xml_list:
    print(item)

for n in imdb_root.findall(f'./movie/production_company/..'):
  print_xml(ET.tostring(n))

In [77]:
# print element
"""
netflix_id = ['netflix_41','netflix_85','netflix_122','netflix_312','netflix_364','netflix_408','netflix_461','netflix_462']
imdb_id = ['imdb_70548','imdb_73841','imdb_77413','imdb_65900','imdb_40872','imdb_72527','imdb_78409','imdb_57491']
streaming_id = ['streaming_6462','streaming_13015','streaming_3882','streaming_4715','streaming_7236','streaming_3435','streaming_4972','streaming_8066']
"""
netflix_id = ['netflix_11313','netflix_11848','netflix_4152','netflix_7082','netflix_9454','netflix_4900','netflix_5432','netflix_3172']
imdb_id = ['imdb_74305','imdb_81898','imdb_33834','imdb_37011','imdb_67779','imdb_81175','imdb_69846','imdb_80344']

#for n, i, s in zip(netflix_id, imdb_id, streaming_id):
for n,i  in zip(netflix_id, imdb_id):
  netflix_note = netflix_root.find(f'.//id[.="{n}"]/..')
  print_xml(ET.tostring(netflix_note))

  imdb_note = imdb_root.find(f'.//id[.="{i}"]/..')
  print_xml(ET.tostring(imdb_note))
"""
  streaming_note = streaming_root.find(f'.//id[.="{s}"]/..')
  print_xml(ET.tostring(streaming_note))
  print('\n')
  """

<movie>
        <id>netflix_11313</id>
        <source>netflix</source>
        <genres>
            <genre>Comedy</genre>
            <genre>Romance</genre>
        </genres>
        <title>Meu Passado Me Condena</title>
        <actors>
            <actor_names>Mi&#225; Mello</actor_names>
            <actor_names>F&#225;bio Porchat</actor_names>
            <actor_names>Inez Viana</actor_names>
            <actor_names>Marcelo Valle</actor_names>
        </actors>
        <release_year>2013</release_year>
        <directors>
            <director>Julia Rezende</director>
        </directors>
        <duration>90</duration>
        <languages>
            <language>Portuguese</language>
        </languages>
        <imdb_votes>2108.0</imdb_votes>
        <production_company>nan</production_company>
        <imdb_score>5.9</imdb_score>
        <writers>
            <writer>Leandro Muniz</writer>
            <writer>Julia Rezende</writer>
            <writer>Mariza Le&#227;o</writer>
 

'\n  streaming_note = streaming_root.find(f\'.//id[.="{s}"]/..\')\n  print_xml(ET.tostring(streaming_note))\n  print(\'\n\')\n  '

### check duplicates

In [ ]:
netflix_tree = ET.parse('data/NEW/netflix.xml')
netflix_root = netflix_tree.getroot()

imdb_tree = ET.parse('data/NEW/imdb.xml')
imdb_root = imdb_tree.getroot()

In [ ]:
imdb_id = 'imdb_32586'
netflix_id = ['netflix_4700', 'netflix_1166', 'netflix_12808']

imdb_note = imdb_root.find(f'.//id[.="{imdb_id}"]/..')
print(ET.tostring(imdb_note))

for id in netflix_id:
  note = netflix_root.find(f'.//id[.="{id}"]/..')
  print(ET.tostring(note))

b'<movie>\n\t\t<id>imdb_32586</id>\n\t\t<title>The Yards</title>\n\t\t<release_year>2000</release_year>\n        <genres>\n            <genre>Romance</genre>\n            <genre>Drama</genre>\n            <genre>Crime</genre>\n        </genres>\n        <actors>\n            <actor_names>Charlize Theron</actor_names>\n            <actor_names>Faye Dunaway</actor_names>\n            <actor_names>Andy Davoli</actor_names>\n            <actor_names>James Caan</actor_names>\n            <actor_names>Robert Montano</actor_names>\n            <actor_names>Steve Lawrence</actor_names>\n            <actor_names>Tomas Milian</actor_names>\n            <actor_names>Victor Argo</actor_names>\n            <actor_names>Chad Aaron</actor_names>\n            <actor_names>Louis Guss</actor_names>\n            <actor_names>Victor Arnold</actor_names>\n            <actor_names>Joaquin Phoenix</actor_names>\n            <actor_names>Mark Wahlberg</actor_names>\n            <actor_names>Ellen Burstyn</act

## post-processing correspondents  - duplicates & imdb 


In [ ]:
# load corr
corr = pd.read_csv('data/netflix_2_imdb_correspondences.csv', header=None)

netflix_duplicates = []
keep = []

# add count column for repeat imdb 
imdb_count = corr[1].value_counts()
corr['count'] = 1
corr['max_sim'] = corr[2]
corr['count1'] = 0 # check duplicates, the number of sim=1

for i, row in corr.iterrows():
  corr.loc[i, 'count'] = imdb_count[row[1]]
  if corr.loc[i, 'count'] > 1:
    # set max_sim
    match = corr[corr[1] == row[1]]
    corr.loc[i, 'max_sim'] = np.max(match[2])
    corr.loc[i, 'count1'] = np.sum(match[2] == 1.0)

# duplicate netflix: count1 >1, then drop the rows
duplicates = corr[corr['count1'] > 1]
corr = corr.drop(duplicates.index)

# records drop: count>1 & sim != max_sim
# re calculate count & max
imdb_count = corr[1].value_counts()

for i, row in corr.iterrows():
  corr.loc[i, 'count'] = imdb_count[row[1]]
  if corr.loc[i, 'count'] > 1:
    match = corr[corr[1] == row[1]]
    corr.loc[i, 'max_sim'] = np.max(match[2])

drop_id = corr[(corr['count'] > 1) & (corr['max_sim'] != corr[2])].index

corr = corr.drop(drop_id)

# : still duplicate netflix, delete and save
imdb_count = corr[1].value_counts()

for i, row in corr.iterrows():
  corr.loc[i, 'count'] = imdb_count[row[1]]

duplicates.append(corr[corr['count'] > 1])
duplicates.to_csv('data/NEW/netflix_duplicates.csv', header=None, index=None)

drop_id = corr[corr['count'] > 1].index
corr = corr.drop(drop_id)

# record the id need to be deleted, if 2 are both 1, duplicate
corr[[0, 1, 2]].to_csv('data/NEW/corr_post.csv', header=None, index=None)

# Gold standard of IR

## preprocessing on dataset


In [47]:
netflix_df = pd.read_csv('data/netflix.csv')
imdb_df = pd.read_csv('data/imdb.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [48]:
# transfer list
list_columns = ['genre', 'actor_names', 'language', 'writer', 'production_company', 'director']

def clean_list(l):
  if pd.isna(l):
    return l
  ret = [item.strip("'\" ") for item in l.strip('[]').split(',') if item != "'None'" ]
  ret = list(set(ret)) # some contains duplicate items, like actors
  if not ret:
    ret = np.nan
  return ret

netflix_df[list_columns] = netflix_df[list_columns].applymap(clean_list)
imdb_df[list_columns] = imdb_df[list_columns].applymap(clean_list)

In [7]:
# title clean - lower and remove puctuation
# new column clean_title
import nltk
from nltk.corpus import stopwords
import string
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

nltk.download('punkt')

netflix_df.title = netflix_df.title.str.lower()
imdb_df.title = imdb_df.title.str.lower()

def clean_title(title):
    words = word_tokenize(title)# tokenize
    punctuations = list(string.punctuation)
    words_preprocessed = [word for word in words if word not in punctuations]
    return ' '.join(words_preprocessed)

netflix_df['clean_title'] = netflix_df.title.apply(clean_title)
imdb_df['clean_title'] = imdb_df.title.apply(clean_title)
display(netflix_df.clean_title)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


0                                 lets fight ghost
1                              how to build a girl
2                                       centigrade
3                                            anne+
4                                            moxie
                           ...                    
15475                       k-pop extreme survival
15476              dreamworks shreks swamp stories
15477    dreamworks happy holidays from madagascar
15478                  dreamworks holiday classics
15479     dreamworks kung fu panda awesome secrets
Name: clean_title, Length: 15480, dtype: object

In [49]:
# clean year
# get closer to release_year

import re
def get_year(year):
    if pd.isna(year):
      return year
    x = re.findall('([\d]{4})', year)
    if x:
      return(x[0])
    else:
      return 0

netflix_df.release_year = netflix_df.release_year.astype(str).apply(get_year).astype(int)
imdb_df.release_year = imdb_df.release_year.astype(str).apply(get_year).astype(int)

## unique match title movies

In [9]:
# title the same and unique & the same year of the same director
inter_df = pd.merge(netflix_df, imdb_df, on='title', how='inner').drop_duplicates(subset=['title'])
ind = []

for i, row in inter_df.iterrows():
  if row['release_year_x'] == row['release_year_y']:
    if isinstance(row['director_x'], list) and isinstance(row['director_y'], list) and len(set(row['director_x']) & set(row['director_y'])) > 0:
      ind.append(i)
      continue

    # at least one actor is the same, treat as a match
    if isinstance(row['actor_names_x'], list) and isinstance(row['actor_names_y'], list) and len(set(row['actor_names_x']) & set(row['actor_names_y'])) > 0:
      ind.append(i)
      
exact_match_df = inter_df.reindex(ind)
print(exact_match_df.shape)
exact_match_df.head()

(2264, 70)


,Unnamed: 0_x,source_x,release_date,genre_x,title,actor_names_x,release_year_x,director_x,duration_x,language_x,imdb_votes_x,production_company_x,imdb_score_x,writer_x,avg_vote_x,country_x,budget_x,original_title_x,reviews_from_critics_x,reviews_from_users_x,title_id_x,country_availability_x,hidden_gem_score_x,image_x,imdb_link_x,netflix_link_x,netflix_release_date_x,poster_x,series_or_movie_x,tags_x,disney_flag_x,hulu_flag_x,netflix_flag_x,prime_video_flag_x,id_x,clean_title_x,Unnamed: 0_y,id_y,source_y,genre_y,actor_names_y,release_year_y,director_y,duration_y,language_y,imdb_votes_y,production_company_y,imdb_score_y,writer_y,avg_vote_y,country_y,budget_y,original_title_y,reviews_from_critics_y,reviews_from_users_y,title_id_y,country_availability_y,hidden_gem_score_y,image_y,imdb_link_y,netflix_link_y,netflix_release_date_y,poster_y,series_or_movie_y,tags_y,disney_flag_y,hulu_flag_y,netflix_flag_y,prime_video_flag_y,clean_title_y
1,2,netflix,2020-08-28,"[Drama, Thriller]",centigrade,"[Genesis Rodriguez, Vincent Piazza]",2020,[Brendan Walsh],90.0,[English],1720.0,NaN,4.3,"[Daley Nixon, Brendan Walsh]",NaN,NaN,16263.0,NaN,NaN,NaN,NaN,['Canada'],6.4,https://occ-0-1081-999.1.nflxso.net/dnm/api/v6...,https://www.imdb.com/title/tt8945942,https://www.netflix.com/watch/81305978,2021-03-04,https://m.media-amazon.com/images/M/MV5BODM2MD...,Movie,['Thrillers'],NaN,NaN,NaN,NaN,netflix_2,centigrade,84911,imdb_84911,imdb,"[Drama, Thriller]","[Vincent Piazza, Genesis Rodriguez]",2020,[Brendan Walsh],98,[English],357,[Manhattan Productions],3.9,[Daley Nixon],3.9,['USA'],NaN,NaN,NaN,NaN,tt8945942,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,centigrade
2,13,netflix,2011-12-25,"[Comedy, Drama]",the stig-helmer story,"[Ida Högberg, Jon Skolmen, Tobias Jacobsson, L...",2011,[Lasse Åberg],90.0,"[English, Swedish]",1666.0,NaN,4.5,[Lasse Åberg],NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Lithuania', 'Poland', 'France', 'Italy', 'Sp...",6.4,https://occ-0-2851-41.1.nflxso.net/dnm/api/v6/...,https://www.imdb.com/title/tt1808587,https://www.netflix.com/watch/81382180,2021-03-03,https://m.media-amazon.com/images/M/MV5BNDU5Y2...,Movie,"['Romantic Dramas', 'Romantic Comedies', 'Dram...",NaN,NaN,NaN,NaN,netflix_13,the stig-helmer story,62161,imdb_62161,imdb,"[Comedy, Drama]","[Tove Edfeldt, Lasse Haldenberg, Filip Arsic-J...",2011,[Lasse Åberg],100,"[English, Swedish]",1597,[Viking Entertainment],4.4,[Lasse Åberg],4.4,"['Sweden', 'USA']",NaN,NaN,NaN,NaN,tt1808587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,the stig-helmer story
16,22,netflix,1989-12-25,[Drama],1939,"[Helene Egelund, Ingvar Hirdwall, Helena Bergs...",1989,[Göran Carmback],180.0,[Swedish],364.0,[AB Svensk Filmindustri],5.5,"[Brasse Brännström, Kjell Sundstedt, Catti Edf...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Lithuania', 'Poland', 'France', 'Italy', 'Sp...",7.6,https://occ-0-2851-41.1.nflxso.net/dnm/api/v6/...,https://www.imdb.com/title/tt0096737,https://www.netflix.com/watch/81382060,2021-03-03,https://m.media-amazon.com/images/M/MV5BZjI3OW...,Movie,"['Swedish Movies', 'Dramas']",NaN,NaN,NaN,NaN,netflix_22,1939,24379,imdb_24379,imdb,[Drama],"[Anders Lönnbro, Per Oscarsson, Ingvar Hirdwal...",1989,[Göran Carmback],193,[Swedish],345,[SVT Drama],5.6,"[Brasse Brännström, Catti Edfeldt]",5.6,['Sweden'],30000000.0,NaN,NaN,NaN,tt0096737,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1939
29,64,netflix,2017-09-15,"[Biography, Drama, Romance, War]",rebel in the rye,"[Kevin Spacey, Nicholas Hoult, Victor Garber, ...",2017,[Danny Strong],90.0,[English],8472.0,[Black Label Media],6.7,"[Danny Strong, Kenneth Slawenski]",NaN,NaN,378294.0,NaN,NaN,NaN,NaN,['South Korea'],3.1,https://occ-0-1361-988.1.nflxso.net/dnm/api/v6...,https://www.imdb.com/title/tt4986134,https://www.netflix.com/watch/80172967,2021-02-28,https://m.media-amazon.com/images/M/MV5BMTUyMD...,Movie,"['Movies Based on Real Life', 'Dramas', 'Movie...",NaN,NaN,NaN,NaN,netflix_64,rebel in the rye,76387,imdb_76387,imdb,"[Biography, Drama, Roma

In [11]:
correspondences = exact_match_df.loc[:, ['id_x', 'id_y']].sample(100)
correspondences['match'] = 'True'
correspondences['corner'] = 0
correspondences.head()
correspondences.to_csv('corr.csv', index=False)

,id_x,id_y,match,corner
5077,netflix_14614,imdb_43053,True,0
2605,netflix_6775,imdb_70687,True,0
1225,netflix_3370,imdb_81730,True,0
5035,netflix_14518,imdb_62247,True,0
2955,netflix_8241,imdb_63623,True,0


## Observations : the characteristic of attributes to select matching attributes


In [50]:
# LOAD data set
gs = pd.read_csv('data/NEW/gs_netflix_imdb.csv', header=None)
gs.columns = ['id_x', 'id_y', 'match']
matching = gs[gs['match'] == True]

matching = pd.merge(matching, netflix_df, how = 'inner', left_on = 'id_x', right_on='id')
matching = pd.merge(matching, imdb_df, how = 'inner', left_on = 'id_y', right_on='id')

matching.head()


,id_x,id_y,match,Unnamed: 0_x,source_x,release_date,genre_x,title_x,actor_names_x,release_year_x,director_x,duration_x,language_x,imdb_votes_x,production_company_x,imdb_score_x,writer_x,avg_vote_x,country_x,budget_x,original_title_x,reviews_from_critics_x,reviews_from_users_x,title_id_x,country_availability_x,hidden_gem_score_x,image_x,imdb_link_x,netflix_link_x,netflix_release_date_x,poster_x,series_or_movie_x,tags_x,disney_flag_x,hulu_flag_x,netflix_flag_x,prime_video_flag_x,id_x,Unnamed: 0_y,id_y,source_y,genre_y,title_y,actor_names_y,release_year_y,director_y,duration_y,language_y,imdb_votes_y,production_company_y,imdb_score_y,writer_y,avg_vote_y,country_y,budget_y,original_title_y,reviews_from_critics_y,reviews_from_users_y,title_id_y,country_availability_y,hidden_gem_score_y,image_y,imdb_link_y,netflix_link_y,netflix_release_date_y,poster_y,series_or_movie_y,tags_y,disney_flag_y,hulu_flag_y,netflix_flag_y,prime_video_flag_y
0,netflix_14614,imdb_43053,True,14614,netflix,2005-05-27,"[Comedy, Family, Animation, Adventure]",Madagascar,"[David Schwimmer, Ben Stiller, Jada Pinkett Sm...",2005,"[Eric Darnell, Tom McGrath]",90.0,"[Portuguese, English, Greek, French, Spanish, ...",367846.0,[DreamWorks SKG],6.9,"[Billy Frolick, Mark Burton, Eric Darnell, Tom...",NaN,NaN,193595521.0,NaN,NaN,NaN,NaN,"['Mexico', 'South Africa', 'Russia', 'Japan', ...",3.0,http://occ-0-1490-1489.1.nflxso.net/dnm/api/v6...,https://www.imdb.com/title/tt0351283,https://www.netflix.com/watch/70021636,2015-04-14,https://images-na.ssl-images-amazon.com/images...,Movie,"['Family Features', 'Animal Tales', 'Family Fe...",NaN,NaN,NaN,NaN,netflix_14614,43053,imdb_43053,imdb,"[Comedy, Animation, Adventure]",Madagascar,"[David Schwimmer, David Cowgill, Sacha Baron C...",2005,"[Eric Darnell, Tom McGrath]",86,"[Portuguese, English, Greek, French, Spanish, ...",357718,[DreamWorks Animation],6.9,"[Mark Burton, Billy Frolick]",6.9,['USA'],75000000.0,NaN,NaN,NaN,tt0351283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,netflix_6775,imdb_70687,True,6775,netflix,2016-08-02,"[Comedy, Family]",Gibby,"[Peyton Meyer, Crystal the Monkey, Shelby Lyon...",2016,[Phil Gorn],90.0,[English],221.0,NaN,5.0,[Greg Lyon],NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Brazil', 'Argentina', 'Mexico', 'Colombia']",7.4,https://occ-0-114-116.1.nflxso.net/dnm/api/v6/...,https://www.imdb.com/title/tt3300932,https://www.netflix.com/watch/81014737,2018-11-15,https://m.media-amazon.com/images/M/MV5BMWRkND...,Movie,"['B-Horror Movies', 'Dramas', 'Children and Fa...",NaN,NaN,NaN,NaN,netflix_6775,70687,imdb_70687,imdb,"[Comedy, Family]",Gibby,"[Luara Hayrapetyan, Shelby Lyon, Vivica A. Fox...",2016,[Phil Gorn],87,[English],216,[Half Moon Films],4.9,[Greg Lyon],4.9,['USA'],1800000.0,NaN,NaN,NaN,tt3300932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,netflix_3370,imdb_81730,True,3370,netflix,2019-05-28,"[Drama, Action, Thriller]",General Commander,"[Billy Ray Gallion, Ron Smoorenburg, Steven Se...",2019,"[Ross W. Clarkson, Philippe Martinez]",90.0,[English],950.0,NaN,3.3,"[Lisa Gabriel, Philippe Martinez]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,['Canada'],6.2,https://occ-0-2430-2433.1.nflxso.net/dnm/api/v...,https://www.imdb.com/title/tt7043534,https://www.netflix.com/watch/81113751,2020-05-02,https://m.media-amazon.com/images/M/MV5BMmZiNm...,Movie,"['Action and Adventure', 'Crime Action and Adv...",NaN,NaN,NaN,NaN,netflix_3370,81730,imdb_81730,imdb,"[Drama, Action, Thriller]",General Commander,"[Billy Ray Gallion, Ron Smoorenburg, Don Gordo...",2019,"[Ross W. Clarkson, Philippe Martinez]",85,[English],780,[Saradan Media],3.4,"[Lisa Gabriel, Philippe Martinez]",3.4,"['UK', 'Philippines', 'Hungary', 'USA']",NaN,NaN,NaN,NaN,tt7043534,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,netflix_14518,imdb_62247,True,14518,netflix,2012-03-28,"[Comedy, Drama, Sport]","Allez, Eddy!","[Jelte Blommaert, Mathias Vergels, Peter Van d...",2012,[Gert Embrechts],90.0,"[French, Dutch]",778.0,NaN,6.4,[Gert Embrechts],NaN,NaN,Na

In [51]:
# compare to see, which columns we can use to compare
match_number = matching.shape[0]

scalar_columns = ['release_year', 'duration', 'imdb_votes', 'imdb_score']
list_columns = ['genre', 'actor_names', 'language', 'writer', 'production_company', 'director']

# exact match
for column in scalar_columns:
  count = (matching[column+'_x'] == matching[column+'_y']).sum()
  print(f'{column}: {count}, {count / match_number}')

# just match one
count_genre = 0
count_actor_names = 0
count_language = 0
count_writer = 0
count_production_company = 0
count_director = 0

for index, row in matching.iterrows():
  for column in list_columns:
    if isinstance(row[column+'_x'], list) and isinstance(row[column+'_y'], list) and len(set(row[column+'_x']).intersection(set(row[column+'_y']))) > 0:
      locals()['count_' + column] += 1

for column in list_columns:
  count = locals()['count_' + column]
  print(f'{column}: {count}, {count / match_number}')


release_year: 218, 0.872
duration: 6, 0.024
imdb_votes: 1, 0.004
imdb_score: 201, 0.804
genre: 250, 1.0
actor_names: 250, 1.0
language: 250, 1.0
writer: 246, 0.984
production_company: 58, 0.232
director: 248, 0.992


There are 10 overlap columns
genre
title
actors
release_year
duration
language
writer
imdb_votes
imdb_score
production_company

By comparing the similarity between those columns of correspondences,

release_year: 218, 0.872

duration: 6, 0.024

imdb_votes: 1, 0.004

imdb_score: 201, 0.804

genre: 250, 1.0

actor_names: 250, 1.0

language: 250, 1.0

writer: 246, 0.984

production_company: 58, 0.232

director: 248, 0.992

We can see, except for "title", we should compare lanuage, actors, genre, writer.


we difinitely should not use duration, imdb_votes, imdb_score, production company

In [ ]:
# missing value percent
netflix_number = netflix_df.shape[0]
imdb_number = imdb_df.shape[0]

for column in list_columns:
  netflix_missing = netflix_df[column].isna().sum()
  imdb_missing = imdb_df[column].isna().sum()
  print(f'\n for {column}, In Netflix, missing rate is {netflix_missing / netflix_number * 100}%')
  print(f'In IMDB, missing rate is {imdb_missing / imdb_number * 100}%')



 for genre, In Netflix, missing rate is 12.94822227276173%
In IMDB, missing rate is 0.0%

 for actor_names, In Netflix, missing rate is 14.593283299219165%
In IMDB, missing rate is 0.08137280712251393%

 for language, In Netflix, missing rate is 14.547797740883937%
In IMDB, missing rate is 0.975277026541895%

 for writer, In Netflix, missing rate is 32.46910772496399%
In IMDB, missing rate is 1.8308881602565634%

 for production_company, In Netflix, missing rate is 70.14631187931165%
In IMDB, missing rate is 5.255726012971783%

 for director, In Netflix, missing rate is 35.615192176483966%
In IMDB, missing rate is 0.10291266783141469%


In [17]:
# get closer to list columns

def closer_to_list_column(column):
  column_s = netflix_df[column].dropna()
  column_netflix = {item for l in column_s.tolist() for item in l}

  column_imdb = {item for l in imdb_df[column].dropna().tolist() for item in l}

  match_column = column_netflix.intersection(column_imdb)

  print(f'there are {len(column_netflix)} {column} in Netflix, {len(column_imdb)} in IMDB, {len(match_column)} in both of dataset')

  print(f'\n {column} in netflix, not in imdb')
  print(column_netflix - column_imdb)

  print('\n in IMDB, not in netflix')
  print(column_imdb - column_netflix)

closer_to_list_column('genre')
closer_to_list_column('language')


there are 28 genre in Netflix, 25 in IMDB, 25 in both of dataset

 genre in netflix, not in imdb
{'Game-Show', 'Short', 'Talk-Show'}

 in IMDB, not in netflix
set()
there are 192 language in Netflix, 267 in IMDB, 188 in both of dataset

 language in netflix, not in imdb
{'Divehi', 'East-Greenlandic', 'Tlingit', 'Chaozhou'}

 in IMDB, not in netflix
{'Ojibwa', 'Tzotzil', 'Bemba', 'Bambara', 'Rotuman', 'Kru', 'Pular', 'Crimean Tatar', 'Athapascan languages', 'Creek', 'Guarani', 'Tigrigna', 'Abkhazian', 'Middle English', 'Russian Sign Language', 'Aymara', 'Visayan', 'Himachali', 'Raeto-Romance', 'Rhaetian', 'Indian Sign Language', 'Mari', 'Hopi', 'Ladakhi', 'Gumatj', 'Kabuverdianu', 'Assyrian Neo-Aramaic', 'Songhay', 'Frisian', 'Ukrainian Sign Language', 'Kashmiri', 'Oriya', 'Ewe', 'Konkani', 'Soninke', 'Ungwatsi', 'Micmac', 'Peul', 'Dyula', 'Kabyle', 'Washoe', 'Dzongkha', 'Mirandese', 'Maithili', 'Minangkabau', 'Tamashek', 'Low German', 'More', 'Arapaho', 'Rajasthani', 'Sindhi', 'Tonga',

In [18]:
closer_to_list_column('director')

there are 7264 director in Netflix, 34707 in IMDB, 5474 in both of dataset

 director in netflix, not in imdb
{'Hideaki Maekawa', 'Da-Won Jeong', 'Chris Eneaji', 'Roger Russell', 'Sebastian Dehnhardt', 'Leandro Aguiari', 'Ismail Basbeth', 'Dominic Palumbo', 'Gisle Sverdrup', 'Tom Ray', 'Seung-ho Choi', 'Dan Kuenster', 'Kareem Tabsch', 'Vaishali Sinha', 'Tyler Lahucik', 'Cassie Quarless', 'Tom Owens', 'Stephan Blinn', 'Yew Kwang Han', 'Chidi Ozieh', 'Titus Tiel Groenestege', 'Phillip L. Dixon II', 'Pin Pin Tan', 'Mark Franchetti', 'Bernie Denk', 'Chris Bolan', 'George LeMaire', 'Jon Freeman', 'Francisco Schultz', 'Kenneth Elvebakk', 'Youngjun Lee', 'JR', 'Sara Dosa', 'Wassim Geagea', 'Pieter Tiddens', 'Izuru Kumasaka', 'Soichi Shimada', 'Kirsten Johnson', 'Michelle Chong', 'Max Lang', 'Anand Kamalakar', 'Debamitra Biswal', 'Otavio Juliano', 'Richard Weston', 'Jim Tshirt', 'One9', 'Dineshkumar Subashchandra', 'Emilio Graziani-Walter', 'Bong Soo Ko', 'Arthur Cary', 'Binu Ulahhannan', 'Ell

There are BLANK in genres of Netflix

In [19]:
# remove the matches
tmp = pd.merge(netflix_df, exact_match_df.loc[:, 'id_x'], left_on='id', right_on='id_x', how='inner').drop(['id_x'], axis=1)
netflix_df = pd.concat([netflix_df, tmp]).drop_duplicates(['id'], keep=False)

tmp = pd.merge(imdb_df, exact_match_df.loc[:, 'id_y'], left_on='id', right_on='id_y', how='inner').drop(['id_y'], axis=1)
imdb_df = pd.concat([imdb_df, tmp]).drop_duplicates(['id'], keep=False)

In [20]:
# add block key  : the first 2 char as key
netflix_df['block_key'] = netflix_df.clean_title.apply(lambda x: x[:2])
imdb_df['block_key'] = imdb_df.clean_title.apply(lambda x: x[:2])

display(pd.Series(netflix_df['block_key']).value_counts())
display(pd.Series(imdb_df['block_key']).value_counts())

th    1882
ma     246
a      221
lo     199
ba     178
      ... 
şa       1
дв       1
3b       1
r.       1
yi       1
Name: block_key, Length: 531, dtype: int64

th    5147
la    4048
il    2933
l'    1668
ma    1662
      ... 
yy       1
2l       1
tn       1
3s       1
         1
Name: block_key, Length: 1084, dtype: int64

## add corner matching cases
For matching, we use title

Year> 5 , remove

give actor, genre, lauguage the same weight

give writer, director less weight cuz more than 30% missing rate

In [ ]:
from Levenshtein import distance

threshold = 0.5
match_list = []

def match_list_attribute(x, y, denominator='min'):
  sim = 0.5 # if NaN, suppose 50% probability the same
  if isinstance(x, list) and isinstance(y, list):
    match_count = len(set(x).intersection(set(y)))
    if match_count == 0:
      return 0

    item_count = 0
    if denominator == 'min':
      item_count = min(len(x), len(y))
    elif denominator == 'max':
      item_count = max(len(x), len(y))
    else:
      item_count = len(x) + len(y) # sum

    sim = match_count / item_count
  return sim

for i, row_x in netflix_df.iterrows():
  for j, row_y in imdb_df[imdb_df['block_key'] == row_x['block_key']].iterrows():
    # match attribute
    # year -- directly pass
    if abs(row_x['release_year'] - row_y['release_year']) > 5:
      continue
    
    # title
    dist = distance(row_x['clean_title'], row_y['clean_title'])
    sim_title = 1 - dist / max(len(row_x['clean_title']), len(row_y['clean_title']))

    if sim_title < 0.5:
      continue

    # actors
    sim_actor = match_list_attribute(row_x['actor_names'], row_y['actor_names'])
    if sim_actor == 0:
      continue
    
    # writer
    sim_writer = match_list_attribute(row_x['writer'], row_y['writer'])
    if sim_writer == 0:
      continue
    
    # director
    sim_director = match_list_attribute(row_x['director'], row_y['director'])
    if sim_director == 0:
      continue

    # genre
    sim_genre = match_list_attribute(row_x['genre'], row_y['genre'], denominator='sum')
    if sim_genre == 0:
      continue

    # language
    sim_language = match_list_attribute(row_x['language'], row_y['language'], denominator='sum')
    if sim_language == 0:
      continue

    # final similarity
    sim = sim_title * 0.6 + sim_actor * 0.1 + sim_director * 0.05 + sim_writer * 0.05 + sim_genre * 0.1 + sim_language * 0.1

    # match the first one--> match the max one
    if sim > threshold:
      match_list.append({
          'id_x': row_x['id'], 'id_y': row_y['id'], 
          'match': 1,
          'corner': 1, 
          'similarity': sim,
          'netflix_title': row_x['title'], 'imdb_title': row_y['title'],
          'netflix_year': row_x['release_year'], 'imdb_year': row_y['release_year'],
          'netflix_actor': row_x['actor_names'], 'imdb_actor': row_y['actor_names'],
          'netflix_director': row_x['director'], 'imdb_director': row_y['director'],
          'netflix_writer': row_x['writer'], 'imdb_writer': row_y['writer'],
          'netflix_genre': row_x['genre'], 'imdb_genre': row_y['genre'],
          'netflix_language': row_x['language'], 'imdb_language': row_y['language']})
      break

print(f'{len(match_list)} movies are matched')


In [28]:
pd.DataFrame(match_list).head()

,id_x,id_y,match,corner,similarity,netflix_title,imdb_title,netflix_year,imdb_year,netflix_actor,imdb_actor,netflix_director,imdb_director,netflix_writer,imdb_writer,netflix_genre,imdb_genre,netflix_language,imdb_language
0,netflix_1,imdb_74014,1,1,0.900000,how to build a girl,how to build a girl,2020,2019,"[Cleo, Beanie Feldstein, Dónal Finn, Paddy Con...","[Sarah Solemani, Beanie Feldstein, Dónal Finn,...",[Coky Giedroyc],[Coky Giedroyc],[Caitlin Moran],[Caitlin Moran],[Comedy],[Comedy],[English],[English]
1,netflix_10,imdb_82625,1,1,0.613077,sunes summer,sune vs. sune,2018,2018,"[Fredrik Hallgren, Tea Stjärne, Baxter Renman,...","[Baxter Renman, Joakim Sikberg, John Österlund...",[Jon Holmberg],[Jon Holmberg],"[Jon Holmberg, Daniella Mendel-Enk, Sara Young...",[Jon Holmberg],"[Fantasy, Comedy, Family]","[Comedy, Family]",[Swedish],[Swedish]
2,netflix_15,imdb_82239,1,1,0.900000,joker,joker,2019,2019,"[Joaquin Phoenix, Zazie Beetz, Frances Conroy,...","[Zazie Beetz, Brett Cullen, Murphy Guyer, Doug...",[Todd Phillips],[Todd Phillips],"[Todd Phillips, Jerry Robinson, Bob Kane, Scot...","[Scott Silver, Todd Phillips]","[Crime, Drama, Thriller]","[Crime, Drama, Thriller]",[English],[English]
3,netflix_19,imdb_21934,1,1,0.637500,false as water,falsk som vatten,1985,1985,"[Sverre Anker Ousdal, Marie Göranzon, Malin Ek...","[Magnus Uggla, Gunilla Olsson, Folke Lind, Mal...",[Hans Alfredson],[Hans Alfredson],[Hans Alfredson],[Hans Alfredson],"[Drama, Thriller]","[Drama, Thriller]",[Swedish],[Swedish]
4,netflix_23,imdb_77308,1,1,0.722222,girls und panzer das finale,girls und panzer the movie,2017,2015,"[Ikumi Nakagami, Mami Ozaki, Ai Kayano, Mai Fu...","[Mai Fuchigami, Mika Kikuchi, Satomi Moriya, Y...",[Tsutomu Mizushima],[Tsutomu Mizushima],[Reiko Yoshida],"[Reiko Yoshida, Fumikane Shimada]","[Comedy, Action, Animation]","[Comedy, Action, Animation]",[Japanese],[Japanese]


In [29]:
pd.DataFrame(match_list).to_csv('match.csv', index=False)

In [ ]:
# remove the matches
correspondences = pd.DataFrame(match_list).reindex(['netflix_id', 'imdb_id'], axis='columns')

tmp = pd.merge(netflix_df, correspondences.loc[:, 'netflix_id'], left_on='id', right_on='netflix_id', how='inner').drop(['netflix_id'], axis=1)
netflix_df = pd.concat([netflix_df, tmp]).drop_duplicates(['id'], keep=False)

tmp = pd.merge(imdb_df, correspondences.loc[:, 'imdb_id'], left_on='id', right_on='imdb_id', how='inner').drop(['imdb_id'], axis=1)
imdb_df = pd.concat([imdb_df, tmp]).drop_duplicates(['id'], keep=False)

In [ ]:
# generate unmatch
total = 240

id_x = netflix_df.sample(total).id.reset_index(drop=True)
id_y = imdb_df.sample(total).id.reset_index(drop=True)

unmatch = pd.DataFrame(list(zip(id_x, id_y)))
unmatch.columns = ['id_x', 'id_y']
unmatch['match'] = 0

unmatch.to_csv('unmatch.csv', index=False)

problems in matching title

- lower case
- punctuation
- language
- typo
- google norm